# Hidden Markov Models (HMM) in Python

HMM can handle a wide range of distributions, such as, discrete tables, Gaussians, and mixtures of Gaussians.

In HMM, it assumes the latent (hidden) discrete multinomial variables $ \{\mathbf{z}_n\} $, which generates the corresponding observation $ \{\mathbf{x}_n\} $. (See below.)<br>
The observers can see only $ \{\mathbf{x}_n\} $, and the model will then be estimated using obervations, $ \{\mathbf{x}_n\} $.

![Hidden Markov Models](images/hmm.png?raw=true)

In HMM, $ p(\mathbf{z}_n|\mathbf{z}_{n-1}) $ is called a **transition probability**, and $ p(\mathbf{x}_n|\mathbf{z}_n) $ is a **emission probability**.

> In this notebook, I denote a scalar variable by normal letter (such as, $ x $), and denote a vector (incl. a matrix) by bold letter (such as, $ \mathbf{x} $).

*back to [Readme](https://github.com/tsmatz/hmm-lds-em-algorithm/)*

##  Sampling in Hidden Markov Models (Generate sample data)

First of all, we'll generate sample data (observations) by using the distribution of Hidden Markov Models (HMM).

As I mentioned above, the distribution of the latent (hidden) variables $ \{\mathbf{z}_n\} $ is discrete, and it then corresponds to a table of transitions.

For sampling, first I'll create a set of latent (hidden) variables, $ \{\mathbf{z}_n\} $, in which it has 3 states (i.e, $ K=3 $) with the following transition probabilities $ p(\mathbf{z}_n|\mathbf{z}_{n-1}) $.

![HMM Discrete Transition](images/transition.png?raw=true)

$$ A = \begin{bmatrix} 0.7 & 0.15 & 0.15 \\ 0.0 & 0.5 & 0.5 \\ 0.3 & 0.35 & 0.35 \end{bmatrix} $$

From now, I'll use the letter $ k \in \{0, 1, 2\} $ for the corresponding 3 states, and I assume $ \mathbf{z}_n = (z_{n,0}, z_{n,1}, z_{n,2}) $, in which $ z_{n,k^{\prime}}=1 $ and $ z_{n,k \neq k^{\prime}}=0 $ in state $ k^{\prime} $.

In [1]:
import numpy as np

np.random.seed(1000)  # To evaluate results

N = 1000

Z = np.array([0])
for n in range(N):
    prev_z = Z[len(Z) - 1]
    if prev_z == 0:
        post_z = np.random.choice(3, size=1, p=[0.7, 0.15, 0.15])
    elif prev_z == 1:
        post_z = np.random.choice(3, size=1, p=[0.0, 0.5, 0.5])
    elif prev_z == 2:
        post_z = np.random.choice(3, size=1, p=[0.3, 0.35, 0.35])
    Z = np.append(Z, post_z)
Z

array([0, 0, 0, ..., 0, 0, 2])

Next I'll create the corresponding observation $ \{\mathbf{x}_n\} $ for sampling.<br>
Here I assume 2-dimensional **Gaussian distribution** $ \mathcal{N}(\mathbf{\mu}_k, \mathbf{\Sigma}_k) $ for emission probabilities $ p(\mathbf{x}_n|\mathbf{z}_n) $, when $ \mathbf{z}_n $ belongs to $ k $. ($ k=0,1,2 $)<br>
In order to simplify, I also assume that parameters $ \mathbf{\mu}_k, \mathbf{\Sigma}_k $ are independent for different components $ k=0, 1, 2 $.

In this example, I set $ \mathbf{\mu}_k, \mathbf{\Sigma}_k $ as follows.

$$ \mathbf{\mu}_0=(16.0, 1.0), \;\; \mathbf{\Sigma}_0 = \begin{bmatrix} 4.0 & 3.5 \\ 3.5 & 4.0 \end{bmatrix} $$

$$ \mathbf{\mu}_1=(1.0, 16.0), \;\; \mathbf{\Sigma}_1 = \begin{bmatrix} 4.0 & 0.0 \\ 0.0 & 1.0 \end{bmatrix} $$

$$ \mathbf{\mu}_2=(-5.0, -5.0), \;\; \mathbf{\Sigma}_2 = \begin{bmatrix} 1.0 & 0.0 \\ 0.0 & 4.0 \end{bmatrix} $$

In [2]:
X = np.empty((0,2))
for z_n in Z:
    if z_n == 0:
        x_n = np.random.multivariate_normal(
            mean=[16.0, 1.0],
            cov=[[4.0,3.5],[3.5,4.0]],
            size=1)
    elif z_n == 1:
        x_n = np.random.multivariate_normal(
            mean=[1.0, 16.0],
            cov=[[4.0,0.0],[0.0,1.0]],
            size=1)
    elif z_n ==2:
        x_n = np.random.multivariate_normal(
            mean=[-5.0, -5.0],
            cov=[[1.0,0.0],[0.0,4.0]],
            size=1)
    X = np.vstack((X, x_n))
X

array([[16.10996367, -0.05478763],
       [18.15392063,  3.77525205],
       [16.73825958,  0.59324625],
       ...,
       [14.2188323 , -1.0984775 ],
       [18.41063372,  5.28130838],
       [-3.64054111, -4.00216984]])

## EM algorithm in Hidden Markov Models (HMM)

Now, using the given observation $ \{ \mathbf{x}_n \} $, let's try to estimate the optimimal parameters in HMM.

When I denote unknown parameters by $ \mathbf{\theta} $, our goal is to get the optimal parameters $ \mathbf{\theta} $ to maximize the following (1).

$$ p(\mathbf{X}|\mathbf{\theta}) = \sum_{\mathbf{Z}} p(\mathbf{X},\mathbf{Z}|\mathbf{\theta}) \;\;\;\;(1) $$

where $ \mathbf{Z} = \{\mathbf{z}_n\} $ and $ \mathbf{X} = \{\mathbf{x}_n\} $

In this example, I use the following parameters as $ \mathbf{\theta} = \{ \mathbf{\pi}, \mathbf{A}, \mathbf{\mu}, \mathbf{\Sigma} \} $.

- $ \pi_k (k \in \{0, 1, 2\}) $ : The possibility (scalar) for component $ k $ in initial latent node $ \mathbf{z}_0 $. ($ \Sigma_k \pi_k = 1 $)
- $ A_{j,k} \; (j, k \in \{0, 1, 2\}) $ : The transition probability (scalar) for the latent variable $ \mathbf{z}_{n-1} $ to $ \mathbf{z}_n $, in which $ \mathbf{z}_{n-1} $ belongs to $ j $ and $ \mathbf{z}_n $ belongs to $ k $. ($ \Sigma_k A_{j,k} = 1 $)
- $ \mathbf{\mu}_k $ : The mean (2-dimensional vector) for Gaussian distribution in emission probabilities $ p(\mathbf{x}_n|\mathbf{z}_n) $ when the latent variable $ \mathbf{z}_n $ belongs to $ k $.
- $ \mathbf{\Sigma}_k $ : The covariance matrix ($ 2 \times 2 $ matrix) for Gaussian distribution in emission probabilities $ p(\mathbf{x}_n|\mathbf{z}_n) $ when the latent variable $ \mathbf{z}_n $ belongs to $ k $.

In (1), the number of parameters will rapidly increase, when the number of states $ K $ increases (in this example, $ K = 3 $). Furthermore it has summation (not multiplication) in distribution (1), and the log likelihood will then lead to complex expression in maximum likelihood estimation (MLE).<br>
Therefore, it will be difficult to directly apply maximum likelihood estimation (MLE) for the expression (1).

> Note : Please see [here](https://tsmatz.wordpress.com/2017/08/30/glm-regression-logistic-poisson-gaussian-gamma-tutorial-with-r/) for the idea of maximum likelihood estimation (MLE).

In practice, the expectation–maximization algorithm (shortly, **EM algorithm**) can often be applied to solve parameters in HMM.<br>
In this example, I'll also apply EM algorithm, instead of MLE.

In EM algorithm for HMM, we start with initial parameters $ \mathbf{\theta}^{old} $, and optimize (find) new $ \mathbf{\theta} $ to maximize the following expression (2).<br>
By repeating this operation, we can expect to reach to the likelihood parameters $ \hat{\mathbf{\theta}} $.

$$ Q(\mathbf{\theta}, \mathbf{\theta}^{old}) = \sum_{\mathbf{Z}} p(\mathbf{Z}|\mathbf{X}, \mathbf{\theta}^{old}) \ln p(\mathbf{X}, \mathbf{Z}|\mathbf{\theta}) \;\;\;\;(2) $$

> Note : For the essential idea of EM algorithm, see Chapter 9 in "[Pattern Recognition and Machine Learning](https://www.microsoft.com/en-us/research/uploads/prod/2006/01/Bishop-Pattern-Recognition-and-Machine-Learning-2006.pdf?ranMID=24542&ranEAID=TnL5HPStwNw&ranSiteID=TnL5HPStwNw-g4zE85KQgCXaCQfYBhtuFQ&epi=TnL5HPStwNw-g4zE85KQgCXaCQfYBhtuFQ&irgwc=1&OCID=AID2200057_aff_7593_1243925&tduid=%28ir__vhvv9m6caokf6nb62oprh029if2xo0rux3ga300300%29%287593%29%281243925%29%28TnL5HPStwNw-g4zE85KQgCXaCQfYBhtuFQ%29%28%29&irclickid=_vhvv9m6caokf6nb62oprh029if2xo0rux3ga300300)" (Christopher M. Bishop, Microsoft)

Now I denote the discrete probability $ p(\mathbf{z}_n|\mathbf{X},\mathbf{\theta}^{old}) $ by $ \gamma(z_{n,k}) \; (k=0,1,2) $, in which $ \gamma(z_{n,k}) $ represents the probability of $ \mathbf{z}_n $ for belonging to $ k $.<br>
I also denote the discrete probability $ p(\mathbf{z}_{n-1}, \mathbf{z}_n | \mathbf{X},\mathbf{\theta}^{old}) $ by $ \xi(z_{n-1,j}, z_{n,k}) \; (j,k=0,1,2) $, in which $ \xi(z_{n-1,j}, z_{n,k}) $ represents the joint probability that $ \mathbf{z}_{n-1} $ belongs to $ j $ and $ \mathbf{z}_n $ belongs to $ k $. 

In Gaussian HMM (in above model), the equation (2) is written as follows, using $ \gamma() $ and $ \xi() $.

$$ Q(\mathbf{\theta}, \mathbf{\theta}^{old}) = \sum_{k=0}^{K-1} \gamma(z_{0,k}) \ln{\pi_k} + \sum_{n=1}^{N-1} \sum_{j=0}^{K-1} \sum_{k=0}^{K-1} \xi(z_{n-1,j},z_{n,k}) \ln{A_{j,k}} + \sum_{n=0}^{N-1} \sum_{k=0}^{K-1} \gamma(z_{n,k}) \ln{p(\mathbf{x}_n|\mathbf{\mu}_k, \mathbf{\Sigma}_k)} \;\;\;\;(3)$$

where

$$ \gamma(\mathbf{z}_n) = p(\mathbf{z}_n|\mathbf{X},\mathbf{\theta}^{old}) $$

$$ \xi(\mathbf{z}_{n-1}, \mathbf{z}_n) = p(\mathbf{z}_{n-1}, \mathbf{z}_n|\mathbf{X},\mathbf{\theta}^{old}) $$

It's known that $ \gamma() $ and $ \xi() $ can be given by the following $ \alpha() $ and $ \beta() $, which are determined recursively. (i.e, We can first determine all $ \alpha() $ and $ \beta() $ recursively, and then we can obtain $ \gamma() $ and $ \xi() $ with known $ \alpha(), \beta() $.)

$$ \gamma(z_{n,k}) = \frac{\alpha(z_{n,k})\beta(z_{n,k})}{\sum_{k=0}^{K-1} \alpha(z_{n,k})\beta(z_{n,k})} $$

$$ \xi(z_{n-1,j},z_{n,k}) = \frac{\alpha(z_{n-1,j})p(\mathbf{x}_n|\mathbf{\mu}_k^{old}, \mathbf{\Sigma}_k^{old})A_{j,k}^{old}\beta(z_{n,k})}{\sum_{j=0}^{K-1} \sum_{k=0}^{K-1} \alpha(z_{n-1,j})p(\mathbf{x}_n|\mathbf{\mu}_k^{old}, \mathbf{\Sigma}_k^{old})A_{j,k}^{old}\beta(z_{n,k})} $$

where all $ \alpha() $ and $ \beta() $ are recursively given by

$$ \alpha(z_{n,k}) = p(\mathbf{x}_n|\mathbf{\mu}_k^{old}, \mathbf{\Sigma}_k^{old}) \sum_{j=0}^{K-1} A_{jk}^{old} \alpha(z_{n-1,j}) $$

$$ \beta(z_{n-1,k}) = \sum_{j=0}^{K-1} A^{old}_{k,j} p(\mathbf{x}_{n}|\mathbf{\mu}_j^{old}, \mathbf{\Sigma}_j^{old}) \beta(z_{n,j}) $$

Now we need the starting condition for recursion, $ \alpha() $ and $ \beta() $, and these are given as follows.

$$ \alpha(z_{0,k}) = \pi_k^{old} p(\mathbf{x}_0|\mathbf{\mu}_k^{old}, \mathbf{\Sigma}_k^{old}) $$

$$ \beta(z_{N-1,k}) = 1 $$

> Note : Here I have showed these properties in Gaussian HMM without any proofs, but you can refer Chapter 13 in "[Pattern Recognition and Machine Learning](https://www.microsoft.com/en-us/research/uploads/prod/2006/01/Bishop-Pattern-Recognition-and-Machine-Learning-2006.pdf?ranMID=24542&ranEAID=TnL5HPStwNw&ranSiteID=TnL5HPStwNw-g4zE85KQgCXaCQfYBhtuFQ&epi=TnL5HPStwNw-g4zE85KQgCXaCQfYBhtuFQ&irgwc=1&OCID=AID2200057_aff_7593_1243925&tduid=%28ir__vhvv9m6caokf6nb62oprh029if2xo0rux3ga300300%29%287593%29%281243925%29%28TnL5HPStwNw-g4zE85KQgCXaCQfYBhtuFQ%29%28%29&irclickid=_vhvv9m6caokf6nb62oprh029if2xo0rux3ga300300)" (Christopher M. Bishop, Microsoft) for details. (In this notebook, I'm using the same notation in this book.)

Once you have got $ \gamma() $ and $ \xi() $, you can get the optimal $ \mathbf{\theta} = \{ \mathbf{\pi}, \mathbf{A}, \mathbf{\mu}, \mathbf{\Sigma} \} $ to maximize (3) as follows by applying Lagrange multipliers.

$$ \pi_k = \frac{\gamma(z_{0,k})}{\sum_{j=0}^{K-1} \gamma(z_{0,j})} $$

$$ A_{j,k} = \frac{\sum_{n=1}^{N-1} \xi(z_{n-1,j},z_{n,k})}{\sum_{l=0}^{K-1} \sum_{n=1}^{N-1} \xi(z_{n-1,j},z_{n,l})} $$

$$ \mathbf{\mu}_k = \frac{\sum_{n=0}^{N-1} \gamma(z_{n,k}) \mathbf{x}_n}{\sum_{n=0}^{N-1} \gamma(z_{n,k})} $$

$$ \mathbf{\Sigma}_k = \frac{\sum_{n=0}^{N-1} \gamma(z_{n,k}) (\mathbf{x}_n-\mathbf{\mu}_k) (\mathbf{x}_n-\mathbf{\mu}_k)^T}{\sum_{n=0}^{N-1} \gamma(z_{n,k})} $$

You repeat this process by replacing $ \mathbf{\theta}^{old} $ with this new $ \mathbf{\theta} $, and you will eventually get the optimal results $ \hat{\mathbf{\theta}} $ to maximize (1).

In practice, $ \alpha() $ and $ \beta() $ will quickly go to zero (because it's recursively multiplied by $ p(\mathbf{x}_n|\mathbf{\mu}_k^{old}, \mathbf{\Sigma}_k^{old}) $ and $ A_{j,k}^{old} $) and it will then exceed the dynamic range of precision in computation, when $ N $ is large.<br>
For this reason, the coefficients, called **scaling factors**, will be introduced to normalize $ \alpha() $ and $ \beta() $ in each step $ n $. (See the following source code.) The scaling factors will be canceled in EM algorithms, however, when you monitor the value of likelihood functions, you'll need to record scaling factors and apply these facotrs. (See Chapter 13 in "[Pattern Recognition and Machine Learning](https://www.microsoft.com/en-us/research/uploads/prod/2006/01/Bishop-Pattern-Recognition-and-Machine-Learning-2006.pdf?ranMID=24542&ranEAID=TnL5HPStwNw&ranSiteID=TnL5HPStwNw-g4zE85KQgCXaCQfYBhtuFQ&epi=TnL5HPStwNw-g4zE85KQgCXaCQfYBhtuFQ&irgwc=1&OCID=AID2200057_aff_7593_1243925&tduid=%28ir__vhvv9m6caokf6nb62oprh029if2xo0rux3ga300300%29%287593%29%281243925%29%28TnL5HPStwNw-g4zE85KQgCXaCQfYBhtuFQ%29%28%29&irclickid=_vhvv9m6caokf6nb62oprh029if2xo0rux3ga300300)" for details.)

## Apply algorithm in Python

## 0. Prerequisites

In [ ]:
!pip3 install numpy
!pip3 install scipy

## 1. Initialize parameters

First, initialize $ \mathbf{\theta} = \{ \pi_k, \mathbf{A}_{j,k}, \mathbf{\mu}_k, \mathbf{\Sigma}_k \} $ as follows.

- $ \pi_0 = 0.3, \pi_1 = 0.3, \pi_2 = 0.4 $
- $ A_{i,j} = 0.4 $ if $ i=j $, and $ A_{i,j} = 0.3 $ otherwise
- $ \mathbf{\mu}_k = (1.0, 1.0) \;\;\; (k = 0,1,2) $
- $ \mathbf{\Sigma}_k = \begin{bmatrix} 1.0 & 0.5 \\ 0.5 & 1.0 \end{bmatrix} \;\;\; (k = 0,1,2) $

In this example, I set the fixed values. However, in practice, K-means will be used to determine initial $ \mathbf{\mu}_k $ and $ \mathbf{\Sigma}_k $, in order to speed up optimization.

In [3]:
# Initialize parameters
theta_old = {
    "pi":[0.3, 0.3, 0.4],
    "A":[[0.4,0.3,0.3],[0.3,0.4,0.3],[0.3,0.3,0.4]],
    "mu":[[1.0,1.0],[1.0,1.0],[1.0,1.0]],
    "Sigma":[
        [[1.0,0.5],[0.5,1.0]],
        [[1.0,0.5],[0.5,1.0]],
        [[1.0,0.5],[0.5,1.0]]
    ]
}

## 2. Get $ \alpha() $ and $ \beta() $

Now I set the starting condition, $ \alpha(z_{0,k}) $. :

$$ \alpha(z_{0,k}) = \pi_k^{old} p(\mathbf{x}_0|\mathbf{\mu}_k^{old}, \mathbf{\Sigma}_k^{old}) $$

And we can recursively obtain all $ \alpha(z_{n,k}) $ as follows.

$$ \alpha(z_{n,k}) = p(\mathbf{x}_n|\mathbf{\mu}_k^{old}, \mathbf{\Sigma}_k^{old}) \sum_{j=0}^{K-1} A_{jk}^{old} \alpha(z_{n-1,j}) $$

As I mentioned above, I also introduce a scaling factor in each steps to prevent the overflow of dynamic range. (Here I don't record these scaling factors.)

In [4]:
from scipy.stats import multivariate_normal

def get_alpha():
    alpha = np.empty((0,3))

    # Get initial alpha_0
    alpha_0 = np.array([])
    for k in range(3):
        p_dist = multivariate_normal(
            mean=theta_old["mu"][k],
            cov=theta_old["Sigma"][k])
        alpha_0 = np.append(alpha_0, theta_old["pi"][k] * p_dist.pdf(X[0]))
    alpha_0 = alpha_0 / alpha_0.sum()  # apply scaling
    alpha = np.vstack((alpha, alpha_0))

    # Get all elements recursively
    for n in range(1, N):
        alpha_n = np.array([])
        for k in range(3):
            p_dist = multivariate_normal(
                mean=theta_old["mu"][k],
                cov=theta_old["Sigma"][k])
            alpha_n = np.append(
                alpha_n,
                p_dist.pdf(X[n]) * sum((theta_old["A"][j][k] * alpha[n-1][j]) for j in range(3)))
        alpha_n = alpha_n / alpha_n.sum()  # apply scaling
        alpha = np.vstack((alpha, alpha_n))

    return alpha

I also set the starting condition, $ \beta(z_{N-1,k}) $. :

$$ \beta(z_{N-1,k}) = 1 $$

And we can recursively obtain all $ \beta(z_{n,k}) $ as follows.

$$ \beta(z_{n-1,k}) = \sum_{j=0}^{K-1} A^{old}_{k,j} p(\mathbf{x}_{n}|\mathbf{\mu}_j^{old}, \mathbf{\Sigma}_j^{old}) \beta(z_{n,j}) $$

As I mentioned above, I also introduce a scaling factor in each steps to prevent the overflow of dynamic range. In practice, the scaling factors can be shared between $ \alpha() $ and $ \beta() $ (and you can then use these shared values for getting values of likelihood function), but in this example, I simply normalize values in each steps.

In [5]:
def get_beta():
    beta_rev = np.empty((0,3))

    # Get initial beta_{N-1}
    beta_last = np.array([1.0, 1.0, 1.0])
    beta_last = beta_last / beta_last.sum()  # apply scaling
    beta_rev = np.vstack((beta_rev, beta_last))

    # Get all elements recursively
    for n in range(1, N):
        beta_rev_n = np.array([])
        for k in range(3):
            beta_rev_n_k = 0
            for j in range(3):
                p_dist = multivariate_normal(
                    mean=theta_old["mu"][j],
                    cov=theta_old["Sigma"][j])
                beta_rev_n_k = theta_old["A"][k][j] * p_dist.pdf(X[n-1]) * beta_rev[n-1][j] + beta_rev_n_k
            beta_rev_n = np.append(beta_rev_n, beta_rev_n_k)
        beta_rev_n = beta_rev_n / beta_rev_n.sum()  # apply scaling
        beta_rev = np.vstack((beta_rev, beta_rev_n))

    # Reverse results
    beta = np.flip(beta_rev, axis=0)
    
    return beta

## 3. Get $ \gamma() $ and $ \xi() $

Now we obtain $ \gamma() $ and $ \xi() $ with previous $ \alpha() $ and $ \beta() $.<br>
First we get $ \gamma() $ as follows. (I note that the value is not normalized.)

$$ \gamma(z_{n,k}) = \frac{\alpha(z_{n,k})\beta(z_{n,k})}{\sum_{k=0}^{K-1} \alpha(z_{n,k})\beta(z_{n,k})} $$

In [6]:
def get_gamma(alpha, beta):
    gamma = np.empty((0,3))

    for n in range(N):
        gamma_n = np.array([])
        for k in range(3):
            gamma_n = np.append(gamma_n, alpha[n][k] * beta[n][k])
        gamma_n = gamma_n / gamma_n.sum()
        gamma = np.vstack((gamma, gamma_n))

    return gamma

Next we also get $ \xi() $ as follows. (I note that the value is not normalized.)

$$ \xi(z_{n-1,j},z_{n,k}) = \frac{\alpha(z_{n-1,j})p(\mathbf{x}_n|\mathbf{\mu}_k^{old}, \mathbf{\Sigma}_k^{old})A_{j,k}^{old}\beta(z_{n,k})}{\sum_{j=0}^{K-1} \sum_{k=0}^{K-1} \alpha(z_{n-1,j})p(\mathbf{x}_n|\mathbf{\mu}_k^{old}, \mathbf{\Sigma}_k^{old})A_{j,k}^{old}\beta(z_{n,k})} $$

In [7]:
def get_xi(alpha, beta):
    xi = np.empty((0,3,3))

    for n in range(1, N):
        xi_n = np.zeros((3,3), dtype=np.float64)
        for j in range(3):
            for k in range(3):
                p_dist = multivariate_normal(
                    mean=theta_old["mu"][k],
                    cov=theta_old["Sigma"][k])
                xi_n[j][k] = alpha[n-1][j] * p_dist.pdf(X[n]) * theta_old["A"][j][k] * beta[n][k]
        xi_n = xi_n / xi_n.sum()
        xi = np.vstack((xi, [xi_n]))

    return xi

## 4. Get new (optimal) parameters $ \mathbf{\theta} $

Finally, get new $ \mathbf{\theta} = \{ \pi_k, A, \mathbf{\mu}, \mathbf{\Sigma} \} $ using previous $ \gamma() $ and $ \xi() $.

$ \pi_k \; (k=0,1,2) $ is given as follows.

$$ \pi_k = \frac{\gamma(z_{0,k})}{\sum_{j=0}^{K-1} \gamma(z_{0,j})} $$

In [8]:
def get_pi_new(gamma):
    pi_new = np.array([])

    denom = sum(gamma[0][j] for j in range(3))
    for k in range(3):
        pi_new = np.append(pi_new, gamma[0][k] / denom)

    return pi_new

$ A_{j,k} \; (j,k=0,1,2) $ is given as follows.

$$ A_{j,k} = \frac{\sum_{n=1}^{N-1} \xi(z_{n-1,j},z_{n,k})}{\sum_{l=0}^{K-1} \sum_{n=1}^{N-1} \xi(z_{n-1,j},z_{n,l})} $$

In [9]:
def get_A_new(xi):
    A_new = np.zeros((3,3), dtype=np.float64)

    for j in range(3):
        for k in range(3):
            denom = 0
            for l in range(3):
                for n in range(1, N):
                    denom = denom + xi[n-1][j][l]
            A_new[j][k] = sum(xi[n-1][j][k] for n in range(1, N)) / denom

    return A_new

$ \mathbf{\mu}_{k} \; (k=0,1,2) $ is given as follows.

$$ \mathbf{\mu}_k = \frac{\sum_{n=0}^{N-1} \gamma(z_{n,k}) \mathbf{x}_n}{\sum_{n=0}^{N-1} \gamma(z_{n,k})} $$

In [10]:
def get_mu_new(gamma):
    mu_new = np.zeros((3,2), dtype=np.float64)

    for k in range(3):
        denom = sum(gamma[n][k] for n in range(N))
        numer_x = sum(gamma[n][k] * X[n][0] for n in range(N))
        mu_new[k][0] = numer_x / denom
        numer_y = sum(gamma[n][k] * X[n][1] for n in range(N))
        mu_new[k][1] = numer_y / denom

    return mu_new

$ \mathbf{\Sigma}_{k} \; (k=0,1,2) $ is given as follows.

$$ \mathbf{\Sigma}_k = \frac{\sum_{n=0}^{N-1} \gamma(z_{n,k}) (\mathbf{x}_n-\mathbf{\mu}_k) (\mathbf{x}_n-\mathbf{\mu}_k)^T}{\sum_{n=0}^{N-1} \gamma(z_{n,k})} $$

In [11]:
def get_Sigma_new(gamma, mu_new):
    Sigma_new = np.empty((0,2,2))

    for k in range(3):
        denom = sum(gamma[n][k] for n in range(N))
        numer = np.zeros((2, 2), dtype=np.float64)
        for n in range(N):
            sub = np.subtract(X[n], mu_new[k])
            sub = np.array([sub])
            sub_t = sub.transpose()
            numer = numer + gamma[n][k] * np.matmul(sub_t, sub)
        Sigma_new = np.vstack((Sigma_new, [numer / denom]))

    return Sigma_new

## 5. Run algorithm

In [12]:
for loop in range(100):
    print("Running iteration {} ...".format(loop + 1), end="\r")
    # Get alpha and beta
    alpha = get_alpha()
    beta = get_beta()
    # Get gamma and xi
    gamma = get_gamma(alpha, beta)
    xi = get_xi(alpha, beta)
    # Get optimized new parameters
    pi_new = get_pi_new(gamma)
    A_new = get_A_new(xi)
    mu_new = get_mu_new(gamma)
    Sigma_new = get_Sigma_new(gamma, mu_new)
    # Replace theta and repeat
    theta_old["pi"] = pi_new
    theta_old["A"] = A_new
    theta_old["mu"] = mu_new
    theta_old["Sigma"] = Sigma_new

print("\nDone")

Running iteration 100 ...
Done


Here is the estimated results for parameters.<br>
I note that the class ($ k $) in results will be transposed and thus I have rearranged its order in the following description.

$$ A^{pred} = \begin{bmatrix} 0.71929825 & 0.06427734 & 0.21642441 \\ 0.0 & 0.23535639 & 0.76464361 \\ 0.32796274 & 0.20301841 & 0.46901885 \end{bmatrix} \;\; A^{label} = \begin{bmatrix} 0.7 & 0.15 & 0.15 \\ 0.0 & 0.5 & 0.5 \\ 0.3 & 0.35 & 0.35 \end{bmatrix} $$

$$ \mathbf{\mu}_0^{pred}=(16.07330508, 0.9698968) \;\; \mathbf{\mu}_0^{label}=(16.0, 1.0), \\ \mathbf{\Sigma}_0^{pred} = \begin{bmatrix} 3.99411283 & 3.47671034 \\ 3.47671034 & 4.06141304 \end{bmatrix} \;\; \mathbf{\Sigma}_0^{label} = \begin{bmatrix} 4.0 & 3.5 \\ 3.5 & 4.0 \end{bmatrix} $$

$$ \mathbf{\mu}_1^{pred}=(1.10889991, 16.13350166) \;\; \mathbf{\mu}_1^{label}=(1.0, 16.0), \\ \mathbf{\Sigma}_1^{pred} = \begin{bmatrix} 4.5466816 & 0.06402755 \\ 0.06402755 & 0.93057687 \end{bmatrix} \;\; \mathbf{\Sigma}_1^{label} = \begin{bmatrix} 4.0 & 0.0 \\ 0.0 & 1.0 \end{bmatrix} $$

$$ \mathbf{\mu}_2^{pred}=(-3.16503151, 1.4672828) \;\; \mathbf{\mu}_2^{label}=(-5.0, -5.0), \\ \mathbf{\Sigma}_2^{pred} = \begin{bmatrix} 9.10191777 & 26.13881586 \\ 26.13881586 & 97.13257096 \end{bmatrix} \;\; \mathbf{\Sigma}_2^{label} = \begin{bmatrix} 1.0 & 0.0 \\ 0.0 & 4.0 \end{bmatrix} $$

In [13]:
np.set_printoptions(suppress=True)
print("A")
print(A_new)
print("Mu")
print(mu_new)
print("Sigma")
print(Sigma_new)

A
[[0.46901885 0.20301841 0.32796274]
 [0.76464361 0.23535639 0.        ]
 [0.21642441 0.06427734 0.71929825]]
Mu
[[-3.16503151  1.4672828 ]
 [ 1.10889991 16.13350166]
 [16.07330508  0.9698968 ]]
Sigma
[[[ 9.10191777 26.13881586]
  [26.13881586 97.13257096]]

 [[ 4.5466816   0.06402755]
  [ 0.06402755  0.93057687]]

 [[ 3.99411283  3.47671034]
  [ 3.47671034  4.06141304]]]
